In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 43.8MB/s]


In [62]:
!pip install kaggle

In [63]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m


 99% 1.42G/1.44G [00:22<00:00, 111MB/s] 
100% 1.44G/1.44G [00:22<00:00, 68.4MB/s]


In [64]:
!7z e fasttext-crawl-300d-2m.zip -o/content -r



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1545551987 bytes (1474 MiB)

Extracting archive: fasttext-crawl-300d-2m.zip
--
Path = fasttext-crawl-300d-2m.zip
Type = zip
Physical Size = 1545551987

  0%      0% - crawl-300d-2M.vec                          1% - crawl-300d-2M.vec                          2% - crawl-300d-2M.vec                          3% - crawl-300d-2M.vec                          4% - crawl-300d-2M.vec                          5% - crawl-300d-2M.vec                        

In [2]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [65]:
# Reading fast text vectors in python: https://stackoverflow.com/a/38230349/4084039
def fasttextModel(gloveFile):
    print ("Loading Fasttext Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}#for storing word and the corresponding embedding vector for that word
    for line in f:
        splitLine = line.split()#splitting the line and storing it in a list
        word = splitLine[0]#getting the first element and storing it in word
        embedding = np.array([float(val) for val in splitLine[1:]])#obtaining corresponding vector for that word
        model[word] = embedding#storing word as key and embedding vector for that word as value
    print ("Done.",len(model)," words loaded!")
    return model
model = fasttextModel('/content/crawl-300d-2M.vec')

Loading Fasttext Model
Done. 2000000  words loaded!


In [3]:
df=pd.read_csv('preprocessed_data.csv')#reading data into DataFrame

In [4]:
df.head(4)#displaying top 4 datapoints


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [5]:
def preprocess(x):#removing last character
  x=x[:-1]
  return x

In [6]:
df['source']=df['source'].apply(preprocess)#preprocessing source data
df['target']=df['target'].apply(preprocess)#preprocessing target data

In [7]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [8]:
df.shape#shape of DataFrame


(2000, 2)

In [9]:
df=df[df['source'].apply(len)<170]#removing source sentences of length greater than or equal to 170
df=df[df['target'].apply(len)<200]#removing target sentences of length greater than or equal to 200

In [10]:
df.shape#shape of DataFrame


(1990, 2)

In [11]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting the data in the ratio 99:1
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


In [57]:
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')
X_test.to_csv('X_test.csv')
y_test.to_csv('y_test.csv')


Target:

In [13]:
target_tokenizer= Tokenizer()#tokenization on target
target_tokenizer.fit_on_texts(y_train)#fitting on ytrain
target_vocab_size= len(target_tokenizer.word_index) + 1#target vocab size
print(len(target_tokenizer.word_index))

3032


In [14]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting text to integers
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting text to integers

In [15]:
target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding to maxlength

In [16]:
target_padded_docs_train.shape


(1970, 43)

In [17]:
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [18]:
target_padded_docs_test.shape


(20, 43)

Source:

In [19]:
source_tokenizer= Tokenizer()#tokenization on source
source_tokenizer.fit_on_texts(X_train)#fitting to X_train
source_vocab_size= len(source_tokenizer.word_index) + 1#source vocab size
print(len(source_tokenizer.word_index))

3703


In [20]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting text to sequence
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting text to sequence

In [21]:
source_padded_docs_train = pad_sequences(source_encoded_docs_train,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [22]:
source_padded_docs_train.shape

(1970, 43)

In [23]:
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [24]:
source_padded_docs_test.shape

(20, 43)

In [25]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))

In [26]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [27]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))

In [28]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [59]:
import pandas as pd 
pd.DataFrame(source_encoded_docs_train).to_csv("source_encoded_docs_train.csv")
pd.DataFrame(source_encoded_docs_test).to_csv("source_encoded_docs_test.csv")
pd.DataFrame(target_encoded_docs_train).to_csv("target_encoded_docs_train.csv")
pd.DataFrame(target_encoded_docs_test).to_csv("target_encoded_docs_test.csv")


In [69]:
#creating embedding matrix
embedding_matrix = np.zeros((source_vocab_size, 300))
for word, i in source_tokenizer.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [70]:
embedding_matrix.shape

(3704, 300)

Model1:

In [75]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,300,weights=[embedding_matrix],input_length=source_padded_docs_train.shape[1],trainable=False)(input)
lstm1=tf.keras.layers.LSTM(128, return_sequences=True)(embed)  
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(lstm1)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()                                                                                                                               

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 43, 300)           1111200   
_________________________________________________________________
lstm_5 (LSTM)                (None, 43, 128)           219648    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 43, 3033)          391257    
Total params: 1,722,105
Trainable params: 610,905
Non-trainable params: 1,111,200
_________________________________________________________________


In [76]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [77]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 14s 542ms/step - loss: 7.9000 - accuracy: 0.3258 - val_loss: 6.3978 - val_accuracy: 0.6814
Epoch 2/50
2/2 [==============================] - 0s 193ms/step - loss: 5.9089 - accuracy: 0.6740 - val_loss: 4.0366 - val_accuracy: 0.6814
Epoch 3/50
2/2 [==============================] - 0s 191ms/step - loss: 3.5780 - accuracy: 0.6740 - val_loss: 2.5258 - val_accuracy: 0.6814
Epoch 4/50
2/2 [==============================] - 0s 191ms/step - loss: 2.7380 - accuracy: 0.6737 - val_loss: 2.9426 - val_accuracy: 0.6814
Epoch 5/50
2/2 [==============================] - 0s 193ms/step - loss: 3.0946 - accuracy: 0.6737 - val_loss: 2.9639 - val_accuracy: 0.6814
Epoch 6/50
2/2 [==============================] - 0s 192ms/step - loss: 3.0431 - accuracy: 0.6737 - val_loss: 2.7388 - val_accuracy: 0.6814
Epoch 7/50
2/2 [==============================] - 0s 196ms/step - loss: 2.7995 - accuracy: 0.6737 - val_loss: 2.5578 - val_accuracy: 0.6814
Epoch 8/50
2/2 [===

In [78]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 0s 219ms/step - loss: 1.4318 - accuracy: 0.7570 - val_loss: 1.5012 - val_accuracy: 0.7558
Epoch 2/50
2/2 [==============================] - 0s 195ms/step - loss: 1.4119 - accuracy: 0.7588 - val_loss: 1.4879 - val_accuracy: 0.7581
Epoch 3/50
2/2 [==============================] - 0s 195ms/step - loss: 1.3912 - accuracy: 0.7616 - val_loss: 1.4809 - val_accuracy: 0.7570
Epoch 4/50
2/2 [==============================] - 0s 197ms/step - loss: 1.3718 - accuracy: 0.7639 - val_loss: 1.4719 - val_accuracy: 0.7593
Epoch 5/50
2/2 [==============================] - 0s 197ms/step - loss: 1.3536 - accuracy: 0.7662 - val_loss: 1.4642 - val_accuracy: 0.7640
Epoch 6/50
2/2 [==============================] - 0s 197ms/step - loss: 1.3356 - accuracy: 0.7684 - val_loss: 1.4547 - val_accuracy: 0.7640
Epoch 7/50
2/2 [==============================] - 0s 195ms/step - loss: 1.3175 - accuracy: 0.7714 - val_loss: 1.4461 - val_accuracy: 0.7663
Epoch 8/50
2/2 [====

In [79]:
x=model.predict(source_padded_docs_test[:1])[0]


In [80]:
#https://machinelearningmastery.com

index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'

' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])


'hey i am still having breakfast <PAD> if you reach there first can help love and me want 12 <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [81]:
print(y_test[:1])


1866    I am still having breakfast. If you reach ther...
Name: target, dtype: object


In [82]:
X_test[:1]


1866    Hey i am still having breakfast eh. If you rea...
Name: source, dtype: object

In [83]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output: ")
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  print(' '.join(y_lst))
  print('>'*180)

Input text: 
Hey i am still having breakfast eh. If you reach there first can help rebecca and me chope seats?
Actual Output: 
I am still having breakfast. If you reach there first can you help me and Rebecca reserve seats?
Predicted Output: 
hey i am still having breakfast if you reach there first can help love and me want 12
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Huh ü take then how i take bus later... Inside got money a not...
Actual Output: 
If you take then how I take bus later? Inside got money or not?
Predicted Output: 
huh you take then how i take bus later then out have a not
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Hi neva worry bout da truth coz the truth will lead me 2 ur

In [84]:
import nltk.translate.bleu_score as bleu

bleu_score=[]
for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  bleu_score.append(bleu.sentence_bleu([b[0].split(),],y_lst))
print(bleu_score)
print("The Average Bleu Score is: ",sum(bleu_score)/20)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.2911892337378932, 0.21142141714303078, 0.14463972129203845, 0.7598356856515925, 0.5081327481546147, 0.5623413251903491, 0.21662732770853732, 0.4063798282013443, 0.7598356856515925, 0.16504659724801518, 0.17795291340072017, 0.30895757752065417, 0.6147881529512643, 0.3769486629893372, 0.1584557519176515, 0.3050975216056289, 0.41545589177443254, 0.28513533990048395, 0.4914498405430853, 0]
The Average Bleu Score is:  0.35798456112911325


Model2:

In [103]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,300,weights=[embedding_matrix],input_length=source_padded_docs_train.shape[1],trainable=False)(input)
lstm1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embed)  
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(lstm1)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 43)]              0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 43, 300)           1111200   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 43, 256)           439296    
_________________________________________________________________
time_distributed_17 (TimeDis (None, 43, 3033)          779481    
Total params: 2,329,977
Trainable params: 1,218,777
Non-trainable params: 1,111,200
_________________________________________________________________


In [104]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [105]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 3s 849ms/step - loss: 7.9556 - accuracy: 0.3262 - val_loss: 6.6151 - val_accuracy: 0.6907
Epoch 2/50
2/2 [==============================] - 0s 241ms/step - loss: 5.7774 - accuracy: 0.6827 - val_loss: 3.0524 - val_accuracy: 0.6814
Epoch 3/50
2/2 [==============================] - 0s 255ms/step - loss: 3.1042 - accuracy: 0.6737 - val_loss: 3.1649 - val_accuracy: 0.6814
Epoch 4/50
2/2 [==============================] - 0s 243ms/step - loss: 3.2320 - accuracy: 0.6737 - val_loss: 2.7467 - val_accuracy: 0.6814
Epoch 5/50
2/2 [==============================] - 0s 246ms/step - loss: 2.7685 - accuracy: 0.6747 - val_loss: 2.4998 - val_accuracy: 0.6837
Epoch 6/50
2/2 [==============================] - 0s 240ms/step - loss: 2.6061 - accuracy: 0.6786 - val_loss: 2.3973 - val_accuracy: 0.6895
Epoch 7/50
2/2 [==============================] - 0s 248ms/step - loss: 2.4415 - accuracy: 0.6799 - val_loss: 2.2410 - val_accuracy: 0.6895
Epoch 8/50
2/2 [====

In [106]:
x=model.predict(source_padded_docs_test[7:8])[0]

In [107]:
index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'

' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])

'how i know i time this one is on <PAD> what <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [108]:
print(y_test[7:8])

128    How I know. Last time this one is on offer.
Name: target, dtype: object


In [109]:
X_test[7:8]

128    How i noe... Last time tis one is on offer wat...
Name: source, dtype: object

In [110]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output: ")
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  print(' '.join(y_lst))
  print('>'*180)

Input text: 
Hey i am still having breakfast eh. If you reach there first can help rebecca and me chope seats?
Actual Output: 
I am still having breakfast. If you reach there first can you help me and Rebecca reserve seats?
Predicted Output: 
hey i am still having day i if you reach there the can help michelle and me buy
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Huh ü take then how i take bus later... Inside got money a not...
Actual Output: 
If you take then how I take bus later? Inside got money or not?
Predicted Output: 
huh you you then how i take bus later and got money a not
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Hi neva worry bout da truth coz the truth will lead me 2 ur heart.

In [111]:
import nltk.translate.bleu_score as bleu

bleu_score=[]
for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  bleu_score.append(bleu.sentence_bleu([b[0].split(),],y_lst))
print(bleu_score)
print("The Average Bleu Score is: ",sum(bleu_score)/20)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.3575297164449809, 0.5828233954152654, 0.14715613818513176, 0.7598356856515925, 0.5081327481546147, 0.5623413251903491, 0.2428662778132657, 0.392814650900513, 0.7598356856515925, 0.3026565453571514, 0.18336673852940621, 0.30895757752065417, 0.6147881529512643, 0.3769486629893372, 0.3114852603245108, 0.32260135189272865, 0.38875142041440197, 0.1788409894677479, 0.4728708045015879, 0]
The Average Bleu Score is:  0.3887301563678047
